<a href="https://colab.research.google.com/github/lucapericlp/lora-latencies/blob/master/LoRA_adapter_merging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers accelerate evaluate datasets peft -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00


In [3]:
import transformers
import accelerate
import peft

print(f"Transformers version: {transformers.__version__}")
print(f"Accelerate version: {accelerate.__version__}")
print(f"PEFT version: {peft.__version__}")

Transformers version: 4.35.2
Accelerate version: 0.24.1
PEFT version: 0.6.2


In [4]:
model_checkpoint = "google/vit-base-patch16-224-in21k"
from datasets import load_dataset

dataset = load_dataset("food101", split="train[:5000]")
labels = dataset.features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

Generating train split:   0%|          | 0/75750 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/25250 [00:00<?, ? examples/s]

In [5]:
from PIL import Image
import requests

url = "https://huggingface.co/datasets/sayakpaul/sample-datasets/resolve/main/beignets.jpeg"
image = Image.open(requests.get(url, stream=True).raw)

In [6]:
from transformers import AutoImageProcessor

repo_name = "sayakpaul/vit-base-patch16-224-in21k-finetuned-lora-food101"
image_processor = AutoImageProcessor.from_pretrained(repo_name)
encoding = image_processor(image.convert("RGB"), return_tensors="pt")

preprocessor_config.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

In [7]:
from peft import LoraConfig

config = {
  "base_model_name_or_path": "google/vit-base-patch16-224-in21k",
  "bias": "none",
  "fan_in_fan_out": False,
  "inference_mode": True,
  "lora_alpha": 16,
  "lora_dropout": 0.1,
  "modules_to_save": [
    "classifier"
  ],
  "peft_type": "LORA",
  "r": 16,
  "target_modules": [
    "query",
    "value"
  ],
  "task_type": None
}
peft_config = LoraConfig(**config)
peft_config.base_model_name_or_path

'google/vit-base-patch16-224-in21k'

In [32]:
from peft import PeftConfig, PeftModel
from transformers import AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained(
    peft_config.base_model_name_or_path,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,  # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)
# Load the LoRA model
inference_model = PeftModel.from_pretrained(model, repo_name, config=peft_config)
inference_model

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModel(
  (base_model): LoraModel(
    (model): ViTForImageClassification(
      (vit): ViTModel(
        (embeddings): ViTEmbeddings(
          (patch_embeddings): ViTPatchEmbeddings(
            (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
          )
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (encoder): ViTEncoder(
          (layer): ModuleList(
            (0-11): 12 x ViTLayer(
              (attention): ViTAttention(
                (attention): ViTSelfAttention(
                  (query): Linear(
                    in_features=768, out_features=768, bias=True
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=16, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_featur

In [33]:
encoding.to("cuda")
inference_model.to("cuda")
inference_model = inference_model.eval()
inference_model

PeftModel(
  (base_model): LoraModel(
    (model): ViTForImageClassification(
      (vit): ViTModel(
        (embeddings): ViTEmbeddings(
          (patch_embeddings): ViTPatchEmbeddings(
            (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
          )
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (encoder): ViTEncoder(
          (layer): ModuleList(
            (0-11): 12 x ViTLayer(
              (attention): ViTAttention(
                (attention): ViTSelfAttention(
                  (query): Linear(
                    in_features=768, out_features=768, bias=True
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=16, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_featur

In [65]:
q_one = inference_model.base_model.model.vit.encoder.layer[0].attention.attention.query
lora_a_linear = q_one.lora_A["default"].weight
lora_b_linear = q_one.lora_B["default"].weight
lora_a_linear.shape, lora_b_linear.shape

(torch.Size([16, 768]), torch.Size([768, 16]))

In [66]:
delta_weights = lora_b_linear @ lora_a_linear
delta_weights.shape

torch.Size([768, 768])

In [67]:
merged_q_one = q_one.weight.data + delta_weights
merged_q_one.shape

torch.Size([768, 768])